### Optimal Portfolio Allocation

An investment universe of the following risky assets with a dependence structure (correlation) is given:

\begin{equation*}

\begin{matrix}
\textbf{Asset} & \boldsymbol\mu & \boldsymbol\sigma & \boldsymbol\omega\\
A & 0.02 & 0.05 & \omega_1\\
B & 0.07 & 0.12 & \omega_2\\
C & 0.15 & 0.17 & \omega_3\\
D & 0.20 & 0.25 & \omega_4
\end{matrix}
\qquad
\qquad
R =
\begin{pmatrix}
1 & 0.3 & 0.3 & 0.3\\
0.3 & 1 & 0.6 & 0.6\\
0.3 & 0.6 & 1 & 0.6\\
0.3 & 0.6 & 0.6 & 1
\end{pmatrix}

\end{equation*}

**Question 2.** Consider optimisation for a tangency portfolio (maximum Sharpe Ratio). 

- Formulate optimisation expression. 
- Formulate Lagrangian function and give its partial derivatives only. 
- For the range of tangency portfolios given by $r_f = 50bps, 100bps, 150bps, 175bps$ optimal compute allocations (ready formula) and $\sigma_\Pi$ . <ins>Present results in a table</ins>. 
    - Plot the efficient frontier in the presence of a risk-free asset for $r_f = 100bps, 175bps$

### Answers and code

The tangency portfolio is invested wholly in risky assets and that maximises the sharpe ratio.

\begin{equation*}
\underset{\boldsymbol\omega}{\arg\max} \quad \frac{\mu - r_f}{\sigma} = \frac{\omega' \mu - r_f}{(\omega' \Sigma \omega)^{\frac{1}{2}}}
\qquad
s.t.
\quad
\omega'1 = 1
\end{equation*}

Form the Langrange function with two multiplier $\lambda$ for a given vector of weights $w$
\begin{equation*}
L(w, \lambda) = (w' \mu - r_f)(w' \Sigma w)^{-\frac{1}{2}} + \lambda(1 - w'1)
\end{equation*}

##### Derivatives

Using a combination of the chain and product rules to obtain the first order derivative

$$
\frac{\partial L}{\partial w} = \mu (w' \Sigma w)^{-\frac{1}{2}} - \frac{1}{2}(w' \mu - r_f)(w' \Sigma w)^{-\frac{3}{2}}2 \Sigma w + \lambda(-1)
$$
simplifying 
$$
\frac{\partial L}{\partial w} = \mu (w' \Sigma w)^{-\frac{1}{2}} - (w' \mu - r_f)(w' \Sigma w)^{-\frac{3}{2}} \Sigma w - \lambda
$$

and the second order derivative

$$
\frac{\partial^2 L}{\partial w^2} = -\frac{1}{2}\mu (w' \Sigma w)^{-\frac{5}{2}}  2\Sigma w - -\frac{3}{2} (w'\Sigma w)^{-frac{5}{2}} 2 \Sigma w \Sigma w + (w' \Sigma w)^{-\frac{3}{2}} \Sigma
$$

simplifying

$$
\frac{\partial^2 L}{\partial w^2} = -\mu (w' \Sigma w)^{-\frac{5}{2}}  \Sigma w + 3 (w'\Sigma w)^{-\frac{5}{2}}  \Sigma w \Sigma w - (w' \Sigma w)^{-\frac{3}{2}} \Sigma
$$

The ready formula to calculate the optimal weights of the tangency portfolio is given by

$$
w_t = \frac{\Sigma^{-1} (\mu - r_f1)}{B - Ar_f}
$$

where A and B are scalars as previously defined in question 1

\begin{equation*}
A = 1\Sigma^{-1} 1'
\end{equation*}

\begin{equation*}
B = \mu' \Sigma^{-1} 1 = 1' \Sigma^{-1} \mu
\end{equation*}

In [9]:
# Import the relevant packages
# Import numpy
import plotly.express as px
import numpy as np
from numpy import *
from numpy.linalg import multi_dot, inv

# Import pandas
import pandas as pd

# Import plotly express
px.defaults.template = 'ggplot2'
px.defaults.width, px.defaults.height = 650, 450
import plotly.io as pio
pio.renderers.default='notebook'

In [10]:
# Set the risk free rates
r = [0.005, 0.01, 0.015, 0.0175]

# Set the matrices for mu and sigma and 1
mu = np.array([0.02, 0.07, 0.15, 0.2]).reshape((4, 1))
sigma = np.array([0.05, 0.12, 0.17, 0.25]).reshape((4, 1))
R = np.array([[1, 0.3, 0.3, 0.3], [0.3, 1, 0.6, 0.6], [
             0.3, 0.6, 1, 0.6], [0.3, 0.6, 0.6, 1]]).reshape(4, 4)
one = np.ones((4,1))

# Compute the covariance matrix usings SRS where S = the diagonal standard deviation matrix
S = np.diag(sigma.flatten())
covariance_matrix = multi_dot([S, R, S])

# Computer the inverse covariance matrix
inverse_covariance_matrix = inv(covariance_matrix)

# Compute A and B
A = multi_dot((one.T,inverse_covariance_matrix,one))
B = multi_dot((one.T,inverse_covariance_matrix,mu))

In [14]:
# For each risk free rate, compute the optimal weights and add them to a dataframe
df = pd.DataFrame([], columns=['rf', 'w_1', 'w_2', 'w_3', 'w_4', 'sigma_pi'])
for rate in r:
    weights = (inverse_covariance_matrix * (mu - rate*one))/(B - A*rate)
    print((inverse_covariance_matrix * (mu - rate*one)))

[[ 6.83937824 -0.38860104 -0.27430661 -0.1865285 ]
 [-1.68393782  8.30274899 -2.10492228 -1.43134715]
 [-2.6516306  -4.69559585  9.22871434 -2.25388601]
 [-2.42487047 -4.29404145 -3.03108808  5.7388601 ]]
[[ 4.55958549 -0.25906736 -0.18287108 -0.12435233]
 [-1.55440415  7.66407599 -1.94300518 -1.32124352]
 [-2.56019506 -4.53367876  8.91048282 -2.1761658 ]
 [-2.3626943  -4.18393782 -2.95336788  5.59170984]]
[[ 2.27979275 -0.12953368 -0.09143554 -0.06217617]
 [-1.42487047  7.02540299 -1.78108808 -1.2111399 ]
 [-2.46875952 -4.37176166  8.59225129 -2.0984456 ]
 [-2.30051813 -4.0738342  -2.87564767  5.44455959]]
[[ 1.13989637 -0.06476684 -0.04571777 -0.03108808]
 [-1.36010363  6.70606649 -1.70012953 -1.15608808]
 [-2.42304176 -4.29080311  8.43313552 -2.05958549]
 [-2.26943005 -4.01878238 -2.83678756  5.37098446]]
